In [1]:
import jieba
import numpy as np
import os

In [2]:
from os import listdir
from os.path import isfile, join,splitext
t_dir = 'dataTrainComplete'
txt_fnames = [splitext(f)[0] for f in listdir(t_dir) if isfile(join(t_dir, f))] #Article filenames

In [3]:
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()

pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()
#Keywords split by lines, keyword with more than one entry will be on the same line

In [4]:
from itertools import chain
import csv
vector_dict = {}
word_list = []
#Keyword lookup with keyword as key and vector index as value
for idx,line in enumerate(chain(crop_line_sep,pest_line_sep,chem_line_sep)):
    l = line.split(',')
    for word in l:
        #Some line will have more than one entry, which should have the same vector index
        if(word == ''):continue
        jieba.add_word(word) #Each keyword is added to jieba
        vector_dict[word] = l[0] # same meaning of different word will have same key number

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.312 seconds.
Prefix dict has been built succesfully.


In [5]:
# adding user dictionary
userdict = ['多變', '溫差', '防檢局', '果農', '防範', '颱風', '台中市', '臺中市',
            '發布', '發佈', '復育', '轄區', '臺南區', '開花期', '莫拉克', '桃園市', '新竹縣']
for word in userdict:
    jieba.add_word(word)

In [6]:
# tokenization
import pandas as pd
df = pd.DataFrame([])
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    df = pd.concat([df, pd.Series([jieba.lcut(content, cut_all=False)])], axis = 0)
df.columns = ['tokenization']
df['name'] = txt_fnames
df = df.reset_index(drop=True)

In [7]:
df

,tokenization,name
0,"[梅雨季, 來臨, ，, 文旦, 黑點病, 易, 發生, ，, 請, 注意, 病徵, ，, ...",1
1,"[天氣, 多變, 溫差, 大, ，, 近, 山區, 及, 偏施, 氮肥, 田區, 稻熱病, ...",10
2,"[新聞稿, -, 稻熱病, 進入, 好發, 季節, ，, 防檢局, 籲請, 農友, 加強, ...",1000
3,"[稻熱病, 進入, 好發, 季節, ，, 防檢局, 籲請, 農友, 加強, 防治, \n, ...",1005
4,"[乍暖還寒, ，, 防檢局, 籲請, 農友, 加強, 防治, 稻熱病, \n, 農委會, 防...",1007
5,"[提早, 入梅, ，, 請, 加強, 防範, 水稻, 稻熱病, 發生, \n, 農委會, 防...",1010
6,"[果實蠅, 已, 進入, 發生, 高峰期, ，, 請, 果農, 加強, 防治, \n, 農委...",1011
7,"[稻熱病, 蠢蠢欲動, 、, 防檢局, 籲請, 農友, 加強, 防範, \n, 目前, 水稻...",1015
8,"[莫拉克, 颱風災, 後, 農作物, 復育, 措施, \n, 莫拉克, 颱風, 造成, 的,...",1016
9,"[嘉義市, 已, 發現, 水稻, 稻熱病, 發生, ，, 籲請, 農友, 掌握, 防治, 時...",1023


In [8]:
# normalize the different word but have same meaning into one word
def keyword_normalization(list_to_be_mapped, key_dict):
    return [key_dict.get(a) if key_dict.get(a) else a for a in list_to_be_mapped]

In [9]:
# mapping all the docs
for index, row in df.iterrows():
    row['tokenization'] = keyword_normalization(df.ix[index]['tokenization'], vector_dict)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df

,tokenization,name
0,"[梅雨季, 來臨, ，, 文旦柚, 黑點病, 易, 發生, ，, 請, 注意, 病徵, ，,...",1
1,"[天氣, 多變, 溫差, 大, ，, 近, 山區, 及, 偏施, 氮肥, 田區, 稻熱病, ...",10
2,"[新聞稿, -, 稻熱病, 進入, 好發, 季節, ，, 防檢局, 籲請, 農友, 加強, ...",1000
3,"[稻熱病, 進入, 好發, 季節, ，, 防檢局, 籲請, 農友, 加強, 防治, \n, ...",1005
4,"[乍暖還寒, ，, 防檢局, 籲請, 農友, 加強, 防治, 稻熱病, \n, 農委會, 防...",1007
5,"[提早, 入梅, ，, 請, 加強, 防範, 水稻, 稻熱病, 發生, \n, 農委會, 防...",1010
6,"[果實蠅, 已, 進入, 發生, 高峰期, ，, 請, 果農, 加強, 防治, \n, 農委...",1011
7,"[稻熱病, 蠢蠢欲動, 、, 防檢局, 籲請, 農友, 加強, 防範, \n, 目前, 水稻...",1015
8,"[莫拉克, 颱風災, 後, 農作物, 復育, 措施, \n, 莫拉克, 颱風, 造成, 的,...",1016
9,"[嘉義市, 已, 發現, 水稻, 稻熱病, 發生, ，, 籲請, 農友, 掌握, 防治, 時...",1023


In [11]:
# concat tokenized text with space
df2list = df.tokenization.tolist()
cut_corpus = []
for i in df2list:
    cut_corpus.append(' '.join(i))

In [12]:
len(cut_corpus)

560

In [14]:
cut_corpus[0]

'梅雨季 來臨 ， 文旦柚 黑點病 易 發生 ， 請 注意 病徵 ， 以及 早 加強 防治 措施 。 \n 5 月 已 進入 梅雨 季節 ， 近日 連續 降雨 ， 為 文旦柚 黑點病 開始 感染 的 時機 ， 往年 文旦柚 在 經過 4 - 6 月 的 春雨 及 梅雨季 後 ， 原來 長得 亮麗 的 果實 外表 ， 會 開始 出現 許多 小黑 點 ， 現在 文旦柚 已 開始 進入 中果期 ， 花蓮區 農業 改良場 呼籲 應 注意 防治 。 \n 除 冬季 清園 作業 外 ， 在 4 - 8 月時 應 每月 施用 一 次 56% 貝芬 硫 \x7f 可 濕性 粉劑 800 倍 、 或 22.7% \x7f 硫 \x7f 水懸劑 1000 倍 、 或 80% 鋅錳乃浦 500 倍 、 或 33% 鋅錳乃浦 500 倍 等 政府 核准 登記 使用 之 藥劑 防治 ， 並 依 登記 使用 方法 使用 ， 尤其 雨前 及 雨後 要 特別 加強 防治 ， 若 遇 連續 降雨 時 則 可 利用 間歇 時 分區 進行 施藥 以 即時 達到 防治 效果 。 \n'

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# functions for analysis module

# CountVector
def CountVectorizer_model(data):
    count_vect = CountVectorizer()
    counts = count_vect.fit_transform(data).toarray() # type: np.array
    # names per every features: count_vect.get_feature_names(), type: list
    return count_vect, count_vect.get_feature_names(), counts

# TF-IDF
def TfidfVectorizer_model(data):
    tfidf_vect = TfidfVectorizer()
    tfidfs = tfidf_vect.fit_transform(data).toarray()
    return tfidf_vect, tfidf_vect.get_feature_names(), tfidfs

In [16]:
# for CountVectorizer module to calculate the term frequency
count_vect, count_feature, counts = CountVectorizer_model(cut_corpus)
# for TfidfVectorizer module to convert a collection of raw documents to a matrix of TF-IDF features.
tfidf_vect, tfidf_feature, tfidfs = TfidfVectorizer_model(cut_corpus)

In [17]:
len(tfidfs)

560

In [18]:
len(tfidf_feature)

7561

In [19]:
tfidfs

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.12710562, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.08548095, ..., 0.        , 0.        ,
        0.        ]])

In [65]:
from gensim.models import Word2Vec
from scipy import spatial
## setting
vector_dim = 200
window_size = 5
min_count = 1
training_iter = 20

## model
word2vec_model = Word2Vec(sentences=df2list,
                          size=vector_dim, window=window_size, 
                          min_count=min_count, iter=training_iter)
# index to word
index_to_word = word2vec_model.wv.index2word # len(word2vec_model.wv.index2word) = 8659
# index to vectors
index2vec = word2vec_model.wv.vectors # word2vec_model.wv.vectors.shape = (8659, 100)

def preprocess(s):
    return [i.lower() for i in s.split()]
def get_vector(s):
    temp_value = np.array([])
    for i in preprocess(s):
        try:
            if temp_value.size == 0:
                temp_value = np.array([word2vec_model[i]]).reshape(1,vector_dim)
            else:
                temp_value = np.concatenate([temp_value, word2vec_model[i].reshape(1,vector_dim)], axis=0)
        except KeyError:
            c = 0
    values = np.sum(temp_value, axis=0)
    return values

word2vec_feature = np.array([])
for i in np.arange(len(cut_corpus)):
    if word2vec_feature.size == 0:
        word2vec_feature = np.array([get_vector(cut_corpus[i])])
    else:
        # concat the two vectors by different columns
        word2vec_feature = np.concatenate((word2vec_feature,np.array([get_vector(cut_corpus[i])])), axis = 0)

# word2vec_feature.shape = (560, 200)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [66]:
word2vec_feature

array([[ -51.087955  ,  -19.82797   ,    8.34267   , ...,    0.95690644,
         -14.417965  ,   28.53353   ],
       [ -61.224552  , -135.74867   ,  -28.876251  , ...,   13.949924  ,
          -9.677287  ,  106.50295   ],
       [-123.81432   , -115.27071   ,   88.83306   , ...,   23.287294  ,
        -186.89291   ,   -5.4177413 ],
       ...,
       [ -69.295654  ,  -77.506676  ,  -13.551053  , ...,   64.85564   ,
         -25.982952  , -100.787895  ],
       [-146.49548   , -168.28397   ,    9.39816   , ...,   98.14657   ,
         -73.30101   ,  -30.817038  ],
       [-174.24149   , -140.50903   ,   17.002472  , ...,   91.8564    ,
         -68.948364  ,  -48.171223  ]], dtype=float32)

In [175]:
labels = {}
pair_list = []
#Deprecated
for fname1 in txt_fnames:
    for fname2 in txt_fnames:
        if(fname1 == fname2):continue
        else:
            labels[(fname1,fname2)] = 0
            pair_list.append((fname1,fname2))
corr_list = open('TrainLabel.csv', "r",encoding='UTF-8-sig')
corr = corr_list.read()
corr_line_sep = corr.splitlines()
#Training label
for line in corr_line_sep[1:]:
    l = line.split(',')
    labels[(l[0],l[1])] = 1
print(len(labels))
print(sum(labels.values()))
print(len(pair_list))

313040
1383
313040


In [176]:
pos_pair_list = []
pos_labels = set() #faster lookup to filter out positive pairs, not used elsewhere
#All the associated article pairs given by train label
for line in corr_line_sep[1:]:
    l = line.split(',')
    pos_pair_list.append((l[0],l[1]))
    pos_labels.add((l[0],l[1]))
neg_pair_list = []
#Other pairs with no association
for fname1 in txt_fnames:
    for fname2 in txt_fnames:
        if(fname1 == fname2):continue
        if((fname1,fname2) not in pos_labels): #Filter out positive pairs
            neg_pair_list.append((fname1,fname2))
print(len(pos_pair_list)+len(neg_pair_list))

313040


In [177]:
import random
# decide which feature you want, tfidf or countvector or others
def construct_input_vector(adapted_vector, tokenized_df, pos_pair_list, neg_pair_list):
    pl = len(pos_pair_list) # Number of positive pairs
    neg_keys = random.sample(neg_pair_list,pl) # same length of negative pairs
    sample_labels = {}
    for key in neg_keys:sample_labels[key] = 0
    for key in pos_pair_list:sample_labels[key] = 1
    sample_pair_list = pos_pair_list + neg_keys
    sample_labels_list = []
    input_vectors = np.array([])
    for i in np.arange(len(sample_pair_list)):
        Test, Ref = sample_pair_list[i] # order of docs
        sample_labels_list.append(sample_labels[(Test,Ref)]) # labels
        Test_vec = adapted_vector[tokenized_df[tokenized_df['name']==Test].index[0]] # from order of doc to find out the order of vectors
        Ref_vec = adapted_vector[tokenized_df[tokenized_df['name']==Ref].index[0]] # same
        temp_vectors = np.concatenate(([Test_vec],[Ref_vec]), axis = 1) # concat the two vectors in same row
        if input_vectors.size == 0:
            input_vectors = temp_vectors
        else:
            input_vectors = np.concatenate((input_vectors,temp_vectors)) # concat the two vectors by different columns
    return input_vectors, sample_labels_list

In [181]:
input_vectors0, sample_labels_list0 = construct_input_vector(tfidfs, df, pos_pair_list, neg_pair_list)
# input_vectors0.shape, len(sample_labels_list0) = (2766, 15122), 2766

In [185]:
input_vectors1, sample_labels_list1 = construct_input_vector(counts, df, pos_pair_list, neg_pair_list)

In [186]:
input_vectors2, sample_labels_list2 = construct_input_vector(word2vec_feature, df, pos_pair_list, neg_pair_list)

In [ ]:
# using these code as embedding features
keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform',
                       embeddings_regularizer=None, activity_regularizer=None,
                       embeddings_constraint=None, mask_zero=False, input_length=None)
model.add(Embedding(index2vec.shape[0], index2vec.shape[1], weights=[index2vec], trainable=False))

## testing

#### preprocessing

In [188]:
datapath = 'Stage_2\dataPublicComplete_s2\dataPublicComplete'
txt_fnames = [splitext(f)[0] for f in listdir(datapath) if isfile(join(datapath, f))]

In [189]:
# tokenization
df = pd.DataFrame([])
for fname in txt_fnames:
    txt = open(datapath+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    df = pd.concat([df, pd.Series([jieba.lcut(content, cut_all=False)])], axis = 0)
df.columns = ['tokenization']
df['name'] = txt_fnames
df = df.reset_index(drop=True)

In [190]:
# mapping all the docs
for index, row in df.iterrows():
    row['tokenization'] = keyword_normalization(df.ix[index]['tokenization'], vector_dict)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [191]:
# concat tokenized text with space
df2list = df.tokenization.tolist()
cut_corpus = []
for i in df2list:
    cut_corpus.append(' '.join(i))

In [192]:
len(cut_corpus)

421

In [193]:
# testing input
# for CountVectorizer module to calculate the term frequency
counts = count_vect.transform(cut_corpus).toarray()
count_feature = count_vect.get_feature_names()
# for TfidfVectorizer module to convert a collection of raw documents to a matrix of TF-IDF features.
tfidfs = tfidf_vect.transform(cut_corpus).toarray()
tfidf_feature = tfidf_vect.get_feature_names()

In [194]:
word2vec_feature = np.array([])
for i in np.arange(len(cut_corpus)):
    if word2vec_feature.size == 0:
        word2vec_feature = np.array([get_vector(cut_corpus[i])])
    else:
        # concat the two vectors by different columns
        word2vec_feature = np.concatenate((word2vec_feature,np.array([get_vector(cut_corpus[i])])), axis = 0)
# word2vec_feature.shape = (421, 200)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
# using these code as embedding features
keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform',
                       embeddings_regularizer=None, activity_regularizer=None,
                       embeddings_constraint=None, mask_zero=False, input_length=None)
model.add(Embedding(index2vec.shape[0], index2vec.shape[1], weights=[index2vec], trainable=False))

In [61]:
labels = {}
pair_list = []
#Deprecated
for fname1 in txt_fnames:
    for fname2 in txt_fnames:
        if(fname1 == fname2):continue
        else:
            labels[(fname1,fname2)] = 0
            pair_list.append((fname1,fname2))

In [68]:
# decide which feature you want, tfidf or countvector or others
def construct_test_input_vector(adapted_vector, tokenized_df, test_pair_list):
    input_vectors = np.array([])
    for i in np.arange(len(test_pair_list)):
        Test, Ref = test_pair_list[i] # order of docs
        Test_vec = adapted_vector[tokenized_df[tokenized_df['name']==Test].index[0]] # from order of doc to find out the order of vectors
        Ref_vec = adapted_vector[tokenized_df[tokenized_df['name']==Ref].index[0]] # same
        temp_vectors = np.concatenate(([Test_vec],[Ref_vec]), axis = 1) # concat the two vectors in same row
        if input_vectors.size == 0:
            input_vectors = temp_vectors
        else:
            input_vectors = np.concatenate((input_vectors,temp_vectors)) # concat the two vectors by different columns
    return input_vectors

In [70]:
a = construct_test_input_vector(tfidfs, df, pair_list)

In [ ]:
a.shape

In [ ]:
a

#### classification

In [ ]:
#

In [ ]:
import csv
with open('val.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Test"]+["Reference"])
    for row in better:  
        spamwriter.writerow(row)